Building a "Research & Summarize" pipeline in LCEL is all about using the pipe operator (|) to pass data through a series of transformations.

In this example, we use RunnablePassthrough to keep our data flowing and StrOutputParser to ensure each step produces clean text for the next link in the chain.

* RunnablePassthrough.assign: This is the "memory" of your chain. It allows you to add new information (like the keywords or search results) to a dictionary while keeping the original user_query available for the final step.

* Separation of Concerns: If the "Relevance Filter" fails, you can tweak just that prompt without breaking the "Synthesis" logic.

* Cost Efficiency: You’ll notice I used gpt-4o-mini for the middle steps. This saves a significant amount of money because you aren't using your most expensive model to do simple keyword extraction.

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

# 1. SETUP MODELS
# We use a cheaper/faster model for filtering and a powerful one for synthesis
fast_llm = ChatOpenAI(model="gpt-4o-mini")
smart_llm = ChatOpenAI(model="gpt-4o")

# 2. STEP 1: KEYWORD EXTRACTOR (The Parser)
keyword_prompt = ChatPromptTemplate.from_template(
    "Extract the 3 most important search keywords from this query: {user_query}"
)
keyword_chain = keyword_prompt | fast_llm | StrOutputParser()

# 3. STEP 2: THE RETRIEVER (Mocked for this example)
# In a real app, this would be a VectorStore or Search Tool
def mock_retriever(keywords):
    return f"Found documents related to: {keywords}. [Doc 1: Technical specs... Doc 2: Market trends...]"

# 4. STEP 3: RELEVANCE FILTER
filter_prompt = ChatPromptTemplate.from_template(
    "Analyze these search results and keep only the technical data: {raw_results}"
)
filter_chain = filter_prompt | fast_llm | StrOutputParser()

# 5. STEP 4: FINAL SYNTHESIS (The Final LLM)
synthesis_prompt = ChatPromptTemplate.from_template(
    "Based on this filtered research: {filtered_info}, write a comprehensive summary for the original query: {user_query}"
)

# --- THE FULL LCEL CHAIN ---
# This "pipes" everything together into one logical flow
# full_research_chain = (
#     {"user_query": RunnablePassthrough()} 
#     | RunnablePassthrough.assign(keywords=keyword_chain) # Extract keywords
#     | RunnablePassthrough.assign(raw_results=lambda x: mock_retriever(x["keywords"])) # Retrieve
#     | RunnablePassthrough.assign(filtered_info=filter_chain) # Filter results
#     | synthesis_prompt # Prepare final prompt
#     | smart_llm # Generate final response
#     | StrOutputParser()
# )

# The Full LCEL Chain with debugging, so you can see each output
from langchain_core.runnables import RunnableLambda #only needed for debug version
def debug_print(x):
    print("\n--- DEBUG STEP ---")
    print(x)
    print("-------------------")
    return x # return input so the chain continues

full_research_chain = (
    {"user_query": RunnablePassthrough()}
    | RunnableLambda(debug_print) # initial input
    | RunnablePassthrough.assign(keywords=keyword_chain)
    | RunnableLambda(debug_print) # see input keywords
    | RunnablePassthrough.assign(raw_results=lambda x: mock_retriever(x["keywords"]))
    | RunnablePassthrough.assign(filtered_info=filter_chain)
    | RunnableLambda(debug_print) # see the full state before synthesis
    | synthesis_prompt
    | smart_llm
    | StrOutputParser()

)
# 6. EXECUTION
result = full_research_chain.invoke("What are the latest advancements in solid-state batteries?")
print(result)


--- DEBUG STEP ---
{'user_query': 'What are the latest advancements in solid-state batteries?'}
-------------------

--- DEBUG STEP ---
{'user_query': 'What are the latest advancements in solid-state batteries?', 'keywords': 'The three most important search keywords from the query are: \n\n1. latest\n2. advancements\n3. solid-state batteries'}
-------------------

--- DEBUG STEP ---
{'user_query': 'What are the latest advancements in solid-state batteries?', 'keywords': 'The three most important search keywords from the query are: \n\n1. latest\n2. advancements\n3. solid-state batteries', 'raw_results': 'Found documents related to: The three most important search keywords from the query are: \n\n1. latest\n2. advancements\n3. solid-state batteries. [Doc 1: Technical specs... Doc 2: Market trends...]', 'filtered_info': 'Based on the provided search results, here is the technical data extracted:\n\n1. **Latest Advancements in Solid-State Batteries**:\n   - Improved energy density metric